In [1]:
import pandas as pd
import numpy as np


## 文本数据
### str对象
- str对象的设计意图

str 对象是定义在 Index 或 Series 上的属性，专门用于逐元素处理文本内容，其内部定义了大量方法，因此对一个序列进行文本处理，首先需要获取其 str 对象。在Python标准库中也有 str 模块，为了使用上的便利，有许多函数的用法 pandas 照搬了它的设计

In [2]:
var = 'abcd'

In [3]:
str.upper(var)

'ABCD'

In [4]:
s = pd.Series(['abcd', 'efg', 'hi'])

In [5]:
s.str

In [6]:
s.str.upper()

0    ABCD
1     EFG
2      HI
dtype: object

[]索引，可以取出某个具体位置的元素

通过对 str 对象使用 [] 索引器，可以完成完全一致的功能，并且如果超出范围则返回缺失值

In [7]:
var[0]

'a'

In [8]:
var[-1: 0: -2]

'db'

In [9]:
s.str[0]

0    a
1    e
2    h
dtype: object

In [10]:
s.str[-1: 0: -2]

0    db
1     g
2     i
dtype: object

In [11]:
s.str[2]

0      c
1      g
2    NaN
dtype: object

### string类型


In [12]:
s = pd.Series([{1: 'temp_1', 2: 'temp_2'}, ['a', 'b'], 0.5, 'my_string'])


In [13]:
s.str[1]

0    temp_1
1         b
2       NaN
3         y
dtype: object

In [14]:
s.astype('string').str[1]

0    1
1    '
2    .
3    y
dtype: string

In [15]:
s = pd.Series(['a'])

In [16]:
s.str.len()

0    1
dtype: int64

In [17]:
s.astype('string').str.len()

0    1
dtype: Int64

In [18]:
s == 'a'

0    True
dtype: bool

In [19]:
s.astype('string') == 'a'

0    True
dtype: boolean

In [20]:
s = pd.Series(['a', np.nan])

In [21]:
s.str.len()

0    1.0
1    NaN
dtype: float64

In [22]:
s.astype('string').str.len()

0       1
1    <NA>
dtype: Int64

In [23]:
s == 'a'

0     True
1    False
dtype: bool

In [24]:
s.astype('string') == 'a'

0    True
1    <NA>
dtype: boolean

In [25]:
s = pd.Series([12, 345, 6789])

In [26]:
s.astype('string').str[1]

0    2
1    4
2    7
dtype: string

## 正则表达式基础
### 一般字符的匹配
正则表达式是一种按照某种正则模式，从左到右匹配字符串中内容的一种工具。对于一般的字符而言，它可以找到其所在的位置，这里为了演示便利，使用了 python 中 re 模块的 findall 函数来匹配所有出现过但不重叠的模式，第一个参数是正则表达式，第二个参数是待匹配的字符串。

In [27]:
import re

In [28]:
re.findall('Apple', 'Apple! This Is an Apple!')

['Apple', 'Apple']

### 元字符基础
-    .     匹配除换行符以外的任意字符
-  [ ]     字符类，匹配方括号中包含的任意字符。
-  [^ ]    否定字符类，匹配方括号中不包含的任意字符
-   *     匹配前面的子表达式零次或多次
-   +     匹配前面的子表达式一次或多次
-   ?     匹配前面的子表达式零次或一次
-  {n,m}    花括号，匹配前面字符至少 n 次，但是不超过 m 次
-  (xyz)    字符组，按照确切的顺序匹配字符xyz。
-   |     分支结构，匹配符号之前的字符或后面的字符
-   \     转义符，它可以还原元字符原来的含义
-   ^     匹配行的开始
-   $     匹配行的结束

In [29]:
re.findall('.', 'abc')

['a', 'b', 'c']

In [30]:
re.findall('[ac]', 'abc')

['a', 'c']

In [31]:
re.findall('[^ac]', 'abc')

['b']

In [32]:
re.findall('[ab]{2}', 'aaaabbbb')

['aa', 'aa', 'bb', 'bb']

In [33]:
re.findall('aaa|bbb', 'aaaabbbb')

['aaa', 'bbb']

In [34]:
re.findall('a\\?|a\*', 'aa?a*a')

['a?', 'a*']

In [35]:
re.findall('a?.', 'abaacadaae')

['ab', 'aa', 'c', 'ad', 'aa', 'e']

### 简写字符集
-  \w     匹配所有字母、数字、下划线: [a-zA-Z0-9_]
-  \W     匹配非字母和数字的字符: [^\w]
-  \d     匹配数字: [0-9]
-  \D     匹配非数字: [^\d]
-  \s     匹配空格符: [\t\n\f\r\p{Z}]
-  \S     匹配非空格符: [^\s]
-  \B     匹配一组非空字符开头或结尾的位置，不代表具体字符

In [36]:
re.findall('.s', 'Apple! This Is an Apple!')

['is', 'Is']

In [37]:
re.findall('\w{2}', '09 8? 7w c_ 9q p@')

['09', '7w', 'c_', '9q']

In [38]:
re.findall('\w\W\B', '09 8? 7w c_ 9q p@')

['8?', 'p@']

In [39]:
re.findall('.\s.', 'Constant dropping wears the stone.')

['t d', 'g w', 's t', 'e s']

In [40]:
re.findall('上海市(.{2,3}区)(.{2,3}路)(\d+号)',
           '上海市黄浦区方浜中路249号 上海市宝山区密山路5号')

[('黄浦区', '方浜中路', '249号'), ('宝山区', '密山路', '5号')]

## 文本处理的五类操作
- 拆分

str.split 能够把字符串的列进行拆分，其中第一个参数为正则表达式，可选参数包括从左到右的最大拆分次数 n ，是否展开为多个列 expand 

In [41]:
s = pd.Series(['上海市黄浦区方浜中路249号',
               '上海市宝山区密山路5号'])

In [42]:
s.str.split('[市区路]')

0    [上海, 黄浦, 方浜中, 249号]
1       [上海, 宝山, 密山, 5号]
dtype: object

In [43]:
s.str.split('[市区路]', n=2, expand=True)

,0,1,2
0,上海,黄浦,方浜中路249号
1,上海,宝山,密山路5号


与其类似的函数是 str.rsplit ，其区别在于使用 n 参数的时候是从右到左限制最大拆分次数。但是当前版本下 rsplit 因为 bug 而无法使用正则表达式进行分割：

In [44]:
s.str.rsplit('[市区路]', n=2, expand=True)

,0
0,上海市黄浦区方浜中路249号
1,上海市宝山区密山路5号


- 2.合并

关于合并一共有两个函数，分别是 str.join 和 str.cat 。 str.join 表示用某个连接符把 Series 中的字符串列表连接起来，如果列表中出现了字符串元素则返回缺失值：

In [45]:
s = pd.Series([['a','b'], [1, 'a'], [['a', 'b'], 'c']])

In [46]:
s.str.join('-')

0    a-b
1    NaN
2    NaN
dtype: object

str.cat 用于合并两个序列，主要参数为连接符 sep 、连接形式 join 以及缺失值替代符号 na_rep ，其中连接形式默认为以索引为键的左连接。


In [47]:
s1 = pd.Series(['a','b'])

In [48]:
s2 = pd.Series(['cat','dog'])

In [49]:
s1.str.cat(s2,sep='-')

0    a-cat
1    b-dog
dtype: object

In [50]:
s2.index = [1, 2]

In [51]:
s1.str.cat(s2, sep='-', na_rep='?', join='outer')

0      a-?
1    b-cat
2    ?-dog
dtype: object

- 3、匹配

str.contains 返回了每个字符串是否包含正则模式的布尔序列

In [52]:
s = pd.Series(['my cat', 'he is fat', 'railway station'])

In [53]:
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

str.startswith 和 str.endswith 返回了每个字符串以给定模式为开始和结束的布尔序列，它们都不支持正则表达式

In [54]:
s.str.startswith('my')

0     True
1    False
2    False
dtype: bool

In [55]:
s.str.endswith('t')

0     True
1     True
2    False
dtype: bool

如果需要用正则表达式来检测开始或结束字符串的模式，可以使用 str.match ，其返回了每个字符串起始处是否符合给定正则模式的布尔序列

In [56]:
s.str.match('m|h')

0     True
1     True
2    False
dtype: bool

In [57]:
s.str[::-1].str.match('ta[f|g]|n')

0    False
1     True
2     True
dtype: bool

In [58]:
s.str.contains('^[m|h]')

0     True
1     True
2    False
dtype: bool

In [59]:
s.str.contains('[f|g]at|n$')

0    False
1     True
2     True
dtype: bool

除了上述返回值为布尔的匹配之外，还有一种返回索引的匹配函数，即 str.find 与 str.rfind ，其分别返回从左到右和从右到左第一次匹配的位置的索引，未找到则返回-1。需要注意的是这两个函数不支持正则匹配，只能用于字符子串的匹配

In [60]:
s = pd.Series(['This is an apple. That is not an apple.'])

In [61]:
s.str.find('apple')

0    11
dtype: int64

In [62]:
s.str.rfind('apple')

0    33
dtype: int64

- 替换

str.replace 和 replace 并不是一个函数，在使用字符串替换时应当使用前者

In [63]:
s = pd.Series(['a_1_b','c_?'])

In [64]:
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

当需要对不同部分进行有差别的替换时，可以利用 子组 的方法，并且此时可以通过传入自定义的替换函数来分别进行处理，注意 group(k) 代表匹配到的第 k 个子组

In [65]:
s = pd.Series(['上海市黄浦区方浜中路249号','上海市宝山区密山路5号',
               '北京市昌平区北农路2号'])

In [66]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'

In [67]:
city = {'上海市': 'Shanghai', '北京市': 'Beijing'}

In [68]:
district = {'昌平区': 'CP District',
            '黄浦区': 'HP District',
            '宝山区': 'BS District'}

In [69]:
road = {'方浜中路': 'Mid Fangbin Road',
        '密山路': 'Mishan Road',
        '北农路': 'Beinong Road'}

In [70]:
def my_func(m):
    str_city = city[m.group(1)]
    str_district = district[m.group(2)]
    str_road = road[m.group(3)]
    str_no = 'No. ' + m.group(4)[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])

In [71]:
s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

In [72]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'

In [73]:
def my_func(m):
    str_city = city[m.group('市名')]
    str_district = district[m.group('区名')]
    str_road = road[m.group('路名')]
    str_no = 'No. ' + m.group('编号')[:-1]
    return ' '.join([str_city,
                     str_district,
                     str_road,
                     str_no])

In [74]:
s.str.replace(pat, my_func, regex=True)

0    Shanghai HP District Mid Fangbin Road No. 249
1           Shanghai BS District Mishan Road No. 5
2           Beijing CP District Beinong Road No. 2
dtype: object

- 5、提取

提取既可以认为是一种返回具体元素（而不是布尔值或元素对应的索引位置）的匹配操作，也可以认为是一种特殊的拆分操作。前面提到的 str.split 例子中会把分隔符去除，这并不是用户想要的效果，这时候就可以用 str.extract 进行提取

In [75]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'

In [76]:
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [77]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'

In [78]:
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [79]:
s = pd.Series(['A135T15,A26S5','B674S2,B25T6'], index = ['my_A','my_B'])


In [80]:
pat = '[A|B](\d+)[T|S](\d+)'

In [81]:
s.str.extractall(pat)

0   1
     match         
my_A 0      135  15
     1       26   5
my_B 0      674   2
     1       25   6

In [82]:
pat_with_name = '[A|B](?P<name1>\d+)[T|S](?P<name2>\d+)'


In [83]:
s.str.extractall(pat_with_name)

name1 name2
     match            
my_A 0       135    15
     1        26     5
my_B 0       674     2
     1        25     6

str.findall 的功能类似于 str.extractall ，区别在于前者把结果存入列表中，而后者处理为多级索引，每个行只对应一组匹配，而不是把所有匹配组合构成列表

In [84]:
s.str.findall(pat)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object

## 常用字符串函数
### 字母型函数
upper, lower, title, capitalize, swapcase 这五个函数主要用于字母的大小写转化，从下面的例子中就容易领会其功能

In [85]:
s = pd.Series(['lower', 'CAPITALS', 'this is a sentence', 'SwApCaSe'])


In [86]:
s.str.upper()

0                 LOWER
1              CAPITALS
2    THIS IS A SENTENCE
3              SWAPCASE
dtype: object

In [87]:
s.str.lower()

0                 lower
1              capitals
2    this is a sentence
3              swapcase
dtype: object

In [88]:
s.str.title()

0                 Lower
1              Capitals
2    This Is A Sentence
3              Swapcase
dtype: object

In [89]:
s.str.capitalize()

0                 Lower
1              Capitals
2    This is a sentence
3              Swapcase
dtype: object

In [90]:
s.str.swapcase()

0                 LOWER
1              capitals
2    THIS IS A SENTENCE
3              sWaPcAsE
dtype: object

### 数值型函数
这里着重需要介绍的是 pd.to_numeric 方法，它虽然不是 str 对象上的方法，但是能够对字符格式的数值进行快速转换和筛选。其主要参数包括 errors 和 downcast 分别代表了非数值的处理模式和转换类型。其中，对于不能转换为数值的有三种 errors 选项， raise, coerce, ignore 分别表示直接报错、设为缺失以及保持原来的字符串

In [91]:
s = pd.Series(['1', '2.2', '2e', '??', '-2.1', '0'])

In [92]:
pd.to_numeric(s, errors='ignore')

0       1
1     2.2
2      2e
3      ??
4    -2.1
5       0
dtype: object

In [93]:
pd.to_numeric(s, errors='coerce')

0    1.0
1    2.2
2    NaN
3    NaN
4   -2.1
5    0.0
dtype: float64

In [94]:
s[pd.to_numeric(s, errors='coerce').isna()]

2    2e
3    ??
dtype: object

### 统计型函数
count 和 len 的作用分别是返回出现正则模式的次数和字符串的长度

In [95]:
s = pd.Series(['cat rat fat at', 'get feed sheet heat'])


In [96]:
s.str.count('[r|f]at|ee')

0    2
1    2
dtype: int64

In [97]:
s.str.len()

0    14
1    19
dtype: int64

### 4、格式型函数

格式型函数主要分为两类，第一种是除空型，第二种时填充型。其中，第一类函数一共有三种，它们分别是 strip, rstrip, lstrip ，分别代表去除两侧空格、右侧空格和左侧空格。这些函数在数据清洗时是有用的，特别是列名含有非法空格的时候

In [98]:
my_index = pd.Index([' col1', 'col2 ', ' col3 '])

In [99]:
my_index.str.strip().str.len()

Int64Index([4, 4, 4], dtype='int64')

In [100]:
my_index.str.rstrip().str.len()

Int64Index([5, 4, 5], dtype='int64')

In [101]:
my_index.str.lstrip().str.len()

Int64Index([4, 5, 5], dtype='int64')

In [102]:
s = pd.Series(['a','b','c'])

In [103]:
s.str.pad(5,'left','*')

0    ****a
1    ****b
2    ****c
dtype: object

In [104]:
s.str.pad(5,'right','*')

0    a****
1    b****
2    c****
dtype: object

In [105]:
s.str.pad(5,'both','*')

0    **a**
1    **b**
2    **c**
dtype: object

In [106]:
# 上述的三种情况可以分别用 rjust, ljust, center 来等效完成，
# 需要注意 ljust 是指右侧填充而不是左侧填充
s.str.rjust(5, '*')

0    ****a
1    ****b
2    ****c
dtype: object

In [107]:
s.str.ljust(5, '*')

0    a****
1    b****
2    c****
dtype: object

In [108]:
s.str.center(5, '*')

0    **a**
1    **b**
2    **c**
dtype: object

在读取 excel 文件时，经常会出现数字前补0的需求，例如证券代码读入的时候会把”000007”作为数值7来处理， pandas 中除了可以使用上面的左侧填充函数进行操作之外，还可用 zfill 来实现。

In [109]:
s = pd.Series([7, 155, 303000]).astype('string')

In [110]:
s.str.pad(6,'left','0')

0    000007
1    000155
2    303000
dtype: string

In [111]:
s.str.rjust(6,'0')

0    000007
1    000155
2    303000
dtype: string

In [112]:
s.str.zfill(6)

0    000007
1    000155
2    303000
dtype: string